In [35]:
import pandas as pd
import time
from datetime import datetime
from fbprophet import Prophet
from flux_query import build_flux_query
from flux_query import dataframe

In [16]:
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
bucket = "04abb7992a07e000"
client = InfluxDBClient(url="http://localhost:9999", token="SVwyqngOyVUM7Ar0DB6nU2rA0il010F1r7wck1a24XF0JDfq15W1Xgz7FTAN3IQk8PkQH9KRoVkVg4hSxT1fIQ==", org="Org")
query_api = client.query_api()
write_api = client.write_api(write_options=SYNCHRONOUS)

In [21]:
query = build_flux_query("pandas ",["activity","weather","test"],{"location":["eur","us"],"season":["fall","summer"]},"-5d","-1h")
query

'from(bucket:"pandas ") |> range(start:-5d, stop:-1h) |> filter(fn: (r) => r._measurement == "activity" or r._measurement == "weather" or r._measurement == "test") |> filter(fn: (r) => r.location == "eur" or r.location == "us" or r.season == "summer")'

In [50]:
test=dataframe('ns',query,"Org",["type","location"])
test.head()

,measurement,tag_key,tag_value,field,value,datetime
0,activity,type,ballet,fun_level,6.0,2019-10-24 18:10:17.529757
1,activity,location,us,fun_level,6.0,2019-10-24 18:10:17.529757
2,activity,type,tennis,fun_level,8.0,2019-10-24 18:10:17.529757
3,activity,location,eur,fun_level,8.0,2019-10-24 18:10:17.529757
4,activity,type,soccer,fun_level,9.0,2019-10-24 18:10:17.529757


In [48]:
def dataframe(precision,query,org,tags):
    result = client.query_api().query(org="Org", query=query)
    raw = []
    for table in result:
        for record in table.records:
            for t in tags:
                raw.append((record.get_measurement(), t, record.values.get(t), record.get_field(), record.get_value(), record.get_time()))
    df=pd.DataFrame(raw, columns=['measurement','tag_key','tag_value','field','value','datetime'], index=None)
    df['datetime'] = df['datetime'].values.astype('<M8'+ '[' + precision + ']')
    return df


In [23]:
result = client.query_api().query(org="Org", query=query)

In [78]:
def lp(df,measurement,tag_key,tag_value,field,value,datetime):
    lines= [str(df[measurement][d]) + "," 
            + str(df[tag_key][d]) + "=" + str(df[tag_value][d]) 
            + " " + str(df[field][d]) + "=" + str(df[value][d]) + "," 
            + " " + str(int(time.mktime(df[datetime][d].timetuple()))) + "000000000" for d in range(len(df))]
    return lines

In [79]:
def write(precision,query,org,tags):
    result = client.query_api().query(org="Org", query=query)
    raw = []
    for table in result:
        for record in table.records:
            for t in tags:
                raw.append((record.get_measurement(), t, record.values.get(t), record.get_field(), record.get_value(), record.get_time()))
    df=pd.DataFrame(raw, columns=['measurement','tag_key','tag_value','field','value','datetime'], index=None)
    df['datetime'] = df['datetime'].values.astype('<M8'+ '[' + precision + ']')
    return df

['activity,type=ballet fun_level=6.0, 1571958617000000000',
 'activity,location=us fun_level=6.0, 1571958617000000000',
 'activity,type=tennis fun_level=8.0, 1571958617000000000',
 'activity,location=eur fun_level=8.0, 1571958617000000000',
 'activity,type=soccer fun_level=9.0, 1571958617000000000',
 'activity,location=eur fun_level=9.0, 1571958617000000000',
 'weather,type=None temperature=81.0, 1571957482000000000',
 'weather,location=us-eastt temperature=81.0, 1571957482000000000',
 'weather,type=None temperature=81.0, 1571958617000000000',
 'weather,location=us temperature=81.0, 1571958617000000000',
 'weather,type=None humidity=70.0, 1571958617000000000',
 'weather,location=eur humidity=70.0, 1571958617000000000',
 'weather,type=None humidity=50.0, 1571958617000000000',
 'weather,location=eur humidity=50.0, 1571958617000000000',
 'activity,type=tag fun=1.0, 1571957482000000000',
 'activity,location=us fun=1.0, 1571957482000000000',
 'activity,type=swimming fun=2.0, 157195748200000

In [ ]:
from influxdb_client import InfluxDBClient, Point, WriteOptions
from influxdb_client.client.write_api import SYNCHRONOUS

_write_client = client.write_api(write_options=WriteOptions(batch_size=1000, 
                                                            flush_interval=10_000,
                                                            jitter_interval=2_000,
                                                            retry_interval=5_000))

_write_client.write("0498523df75cd000", "Org", lines)